In [ ]:
import json

# -----------------------------
# Load Firestore JSON
# -----------------------------
with open("backup.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# -----------------------------
# Mapping: Firestore collections → MySQL tables
# -----------------------------
collection_to_table = {
    "users": "users",
    "targets": "salesTargets",
    "deals": "deals",
    "callbacks": "callbacks",
    "target_progress": "targetProgress",
    "notifications": "notifications"
}

# -----------------------------
# Generate SQL inserts
# -----------------------------
with open("inserts.sql", "w", encoding="utf-8") as sql_file:
    for collection, table in collection_to_table.items():
        if collection not in data:
            continue

        for doc_id, doc_data in data[collection].items():
            fields = []
            values = []
            for key, value in doc_data.items():
                fields.append(f"`{key}`")

                if value is None:
                    values.append("NULL")
                elif isinstance(value, str):
                    safe_value = value.replace("'", "''")  # escape quotes
                    values.append(f"'{safe_value}'")
                else:
                    values.append(str(value))

            insert_sql = f"INSERT INTO {table} ({', '.join(fields)}) VALUES ({', '.join(values)});\n"
            sql_file.write(insert_sql)

print("✅ inserts.sql file created with all data")


✅ inserts.sql file created with all data


In [1]:
# toSQL: Firestore JSON -> MySQL (phpMyAdmin-friendly)
# Generates:
#  - tables.sql  (CREATE TABLE statements)
#  - inserts.sql (INSERT INTO statements)
#
# Notes:
# - Infers schema by scanning all docs per collection
# - Firestore timestamp objects:
#     {"type":"firestore/timestamp/1.0","seconds":..., "nanoseconds":...} -> DATETIME
# - Strings that match YYYY-MM-DD -> DATE
# - Strings that match YYYY-MM-DD HH:MM:SS (or with T, Z) -> DATETIME
# - Booleans -> TINYINT(1)
# - Numbers: if any float -> DOUBLE, else BIGINT
# - Unknown nested objects/arrays -> JSON
# - Adds a synthetic `id` (VARCHAR(191)) column as PRIMARY KEY for the doc ID
# - All column names and table names are backticked
# - Engine=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci

import json
import os
import re
from datetime import datetime, timezone

# -----------------------------
# Paths
# -----------------------------
BASE_DIR = os.path.dirname(os.path.abspath(__file__)) if "__file__" in globals() else os.getcwd()
INPUT_JSON = os.path.join(BASE_DIR, "backup2.json")  # Ensure this file exists in database/
TABLES_SQL = os.path.join(BASE_DIR, "tables.sql")
INSERTS_SQL = os.path.join(BASE_DIR, "inserts.sql")

# -----------------------------
# Helpers: type detection
# -----------------------------
DATE_RE = re.compile(r"^\d{4}-\d{2}-\d{2}$")
DATETIME_RE = re.compile(
    r"^\d{4}-\d{2}-\d{2}[ T]\d{2}:\d{2}:\d{2}(?:\.\d+)?(?:Z|[+-]\d{2}:\d{2})?$",
    re.IGNORECASE
)

def is_firestore_timestamp(obj):
    return (
        isinstance(obj, dict)
        and obj.get("type") == "firestore/timestamp/1.0"
        and "seconds" in obj
    )

def normalize_firestore_timestamp(obj):
    # Convert seconds (UTC) to 'YYYY-MM-DD HH:MM:SS'
    try:
        dt = datetime.fromtimestamp(int(obj["seconds"]), tz=timezone.utc)
        return dt.strftime("%Y-%m-%d %H:%M:%S")
    except Exception:
        return None

def detect_scalar_type(value):
    # Returns one of: 'null', 'bool', 'int', 'float', 'date', 'datetime', 'string', 'timestamp', 'json'
    if value is None:
        return "null"
    if is_firestore_timestamp(value):
        return "timestamp"
    if isinstance(value, bool):
        return "bool"
    if isinstance(value, int):
        return "int"
    if isinstance(value, float):
        return "float"
    if isinstance(value, str):
        s = value.strip()
        if DATE_RE.match(s):
            return "date"
        if DATETIME_RE.match(s):
            return "datetime"
        return "string"
    # arrays or objects (non-timestamp): treat as JSON
    if isinstance(value, (list, dict)):
        return "json"
    # fallback
    return "string"

def merge_types(existing, new):
    # Merge detected types for a field across documents
    # Priority order for final SQL typing:
    #  - If any json -> JSON
    #  - Else if any timestamp -> DATETIME
    #  - Else if any datetime -> DATETIME
    #  - Else if any date -> DATE (unless datetime present)
    #  - Else if any float -> DOUBLE
    #  - Else if any int -> BIGINT
    #  - Else if any bool -> TINYINT(1)
    #  - Else string -> TEXT
    if existing is None:
        return {new}
    existing.add(new)
    return existing

def choose_sql_type(types_set, field_name=""):
    types = types_set or set(["string"])
    # JSON dominates
    if "json" in types:
        return "JSON"
    if "timestamp" in types or "datetime" in types:
        return "DATETIME"
    if "date" in types and "datetime" not in types and "timestamp" not in types:
        return "DATE"
    if "float" in types:
        return "DOUBLE"
    if "int" in types:
        return "BIGINT"
    if "bool" in types:
        return "TINYINT(1)"
    # Fallback to TEXT for strings/mixed
    # You can customize some known fields to VARCHAR(191)
    if field_name in {"email", "username", "name", "team", "role", "sales_team", "service_tier", "stage", "priority", "country", "customer_name", "phone_number", "DealID", "device_id", "device_key", "created_by", "created_by_id", "SalesAgentID", "ClosingAgentID", "sales_agent", "closing_agent", "sales_agent_norm", "closing_agent_norm", "managerId", "managerName", "agentId", "agentName", "period"}:
        return "VARCHAR(191)"
    return "TEXT"

def sql_escape_string(s):
    # Escape single quotes by doubling them, and backslashes
    return s.replace("\\", "\\\\").replace("'", "''")

def value_to_sql_literal(value, sql_type):
    if value is None:
        return "NULL"
    if sql_type == "TINYINT(1)":
        # Boolean or small int
        if isinstance(value, bool):
            return "1" if value else "0"
        return "1" if str(value).lower() == "true" else ("0" if str(value).lower() == "false" else str(int(value)))
    if sql_type in ("BIGINT", "DOUBLE"):
        try:
            if isinstance(value, bool):
                return "1" if value else "0"
            if sql_type == "BIGINT":
                return str(int(value))
            return str(float(value))
        except Exception:
            # fallback to quoted string
            return f"'{sql_escape_string(str(value))}'"
    if sql_type == "DATE":
        if isinstance(value, str) and DATE_RE.match(value.strip()):
            return f"'{value.strip()}'"
        # Best-effort format
        return "NULL"
    if sql_type == "DATETIME":
        if is_firestore_timestamp(value):
            norm = normalize_firestore_timestamp(value)
            return f"'{norm}'" if norm else "NULL"
        if isinstance(value, str):
            s = value.strip().replace("T", " ").replace("Z", "")
            # Attempt parse
            try:
                # Normalize to 'YYYY-MM-DD HH:MM:SS'
                dt = datetime.fromisoformat(s)
                return f"'{dt.strftime('%Y-%m-%d %H:%M:%S')}'"
            except Exception:
                # If already "YYYY-MM-DD HH:MM:SS"
                if DATETIME_RE.match(s):
                    # If it includes offset, we could parse; otherwise, accept as is
                    parts = s.split(".")[0]
                    return f"'{parts}'" if " " in parts else f"'{parts} 00:00:00'"
        return "NULL"
    if sql_type == "JSON":
        try:
            return f"'{sql_escape_string(json.dumps(value, ensure_ascii=False))}'"
        except Exception:
            return "NULL"
    # TEXT and VARCHAR
    s = str(value)
    return f"'{sql_escape_string(s)}'"

# -----------------------------
# Load data
# -----------------------------
with open(INPUT_JSON, "r", encoding="utf-8") as f:
    data = json.load(f)

# Collections may be dynamic; we’ll iterate all top-level keys that are dicts
collections = {k: v for k, v in data.items() if isinstance(v, dict)}

# -----------------------------
# Infer schemas per collection
# -----------------------------
schemas = {}  # {table_name: {field_name: sql_type}}
order_of_fields = {}  # preserve a deterministic order
for coll_name, docs in collections.items():
    # Table name = collection name as-is
    table = coll_name
    field_types = {}       # {field_name: set(types)}
    fields_seen_order = [] # deterministic order
    
    # We will always include an 'id' column for the document ID
    field_types["id"] = {"string"}  # id as string
    fields_seen_order.append("id")
    
    for doc_id, doc_data in docs.items():
        if not isinstance(doc_data, dict):
            # Skip malformed items
            continue
        # Merge keys
        for k, v in doc_data.items():
            t = detect_scalar_type(v)
            if k not in field_types:
                field_types[k] = set()
                fields_seen_order.append(k)
            field_types[k] = merge_types(field_types[k], t)
    
    # Choose SQL types
    sql_field_types = {}
    for k in fields_seen_order:
        sql_field_types[k] = choose_sql_type(field_types.get(k), field_name=k)
    
    # Ensure 'id' is VARCHAR(191)
    sql_field_types["id"] = "VARCHAR(191)"
    
    schemas[table] = sql_field_types
    order_of_fields[table] = fields_seen_order

# -----------------------------
# Generate CREATE TABLE statements
# -----------------------------
create_statements = []
for table, fields in schemas.items():
    cols = []
    for fname in order_of_fields[table]:
        sql_type = fields[fname]
        nullability = "NULL"  # allow NULLs by default
        # You can mark id as NOT NULL
        if fname == "id":
            nullability = "NOT NULL"
        cols.append(f"  `{fname}` {sql_type} {nullability}")
    cols.append("  PRIMARY KEY (`id`)")
    create_sql = (
        f"CREATE TABLE IF NOT EXISTS `{table}` (\n"
        + ",\n".join(cols)
        + "\n) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;"
    )
    create_statements.append(create_sql)

with open(TABLES_SQL, "w", encoding="utf-8") as f:
    f.write("-- Auto-generated by toSQL notebook\n\n")
    f.write("SET NAMES utf8mb4;\nSET time_zone = '+00:00';\n\n")
    for stmt in create_statements:
        f.write(stmt + "\n\n")

# -----------------------------
# Generate INSERT statements
# -----------------------------
with open(INSERTS_SQL, "w", encoding="utf-8") as f:
    f.write("-- Auto-generated by toSQL notebook\n\n")
    for table, docs in collections.items():
        fields_order = order_of_fields[table]
        # Emit rows
        for doc_id, doc_data in docs.items():
            if not isinstance(doc_data, dict):
                continue
            values_sql = []
            for fname in fields_order:
                sql_type = schemas[table][fname]
                if fname == "id":
                    values_sql.append(value_to_sql_literal(doc_id, "VARCHAR(191)"))
                else:
                    raw_val = doc_data.get(fname)
                    values_sql.append(value_to_sql_literal(raw_val, sql_type))
            insert_sql = f"INSERT INTO `{table}` ({', '.join(f'`{c}`' for c in fields_order)}) VALUES ({', '.join(values_sql)});"
            f.write(insert_sql + "\n")

print("✅ Generated:")
print(f" - {os.path.relpath(TABLES_SQL, BASE_DIR)}")
print(f" - {os.path.relpath(INSERTS_SQL, BASE_DIR)}")

✅ Generated:
 - tables.sql
 - inserts.sql
